In [1]:
import tkinter as tk
import time 
import threading 
import simpy
import random
import sys
sys.setrecursionlimit(10000000)

In [ ]:
#clients arrive at a random time and rainbow color  

fenetre = tk.Tk()

canvas = tk.Canvas(fenetre, width=800, height=500, background='black')
rect = canvas.create_rectangle(100, 100, 150, 400,fill='red')
line = canvas.create_line(300,235,300,285, fill='red')
c = []
coords = []
t = []
wait = []
coord_rect = (100,100,150,400)
coord_line = (300,235,300,285)
timeInBank = 15.0
arrInt = 3.0
nb_counter = 4
counter = [True]*nb_counter
dispo = []

class Counter(): 
    def __init__(self): 
        color = 'green'
        dispo = True 
        

for k in range(nb_counter): 
    dispo.append(canvas.create_rectangle(130,250 + 150*(2*k - (nb_counter - 1))/nb_counter,150,270 + 150*(2*k - (nb_counter - 1))/nb_counter,fill='green'))
    
def modif_ind(tab, i): 
    for j in range(i, len(tab)):
        tab[j].i = tab[j].i - 1 
        
def hex_to_RGB(hex): 
    return [int(hex[i:i+2], 16) for i in range(1, 6, 2)]

def RGB_to_hex(RGB): 
    RGB = [int(x) for x in RGB]
    return "#" + "".join(["0{0:x}".format(v) if v < 16 else "{0:x}".format(v) for v in RGB ])

def color_dict(gradient):
    return {"hex":[RGB_to_hex(RGB) for RGB in gradient], "r":[RGB[0] for RGB in gradient], "g":[RGB[1] for RGB in gradient], "b":[RGB[2] for RGB in gradient]}

def linear_gradient(start_hex, finish_hex="#7711AA", n=10000): 
    s = hex_to_RGB(start_hex)
    f = hex_to_RGB(finish_hex)
    RGB_list = [s]
    for t in range(1,n): 
        curr_vector = [int(s[j] + (float(t)/(n - 1))*(f[j] - s[j])) for j in range(3)] 
        RGB_list.append(curr_vector)
    return color_dict(RGB_list)

temp = linear_gradient("#00FFFF")
color = temp['hex']
                   
                   
                   
class myThread(threading.Thread):

    def __init__(self, i):
        threading.Thread.__init__(self)
        self.i = i
        self.going = False
        self.comptoir = -1
        self.deplacement = 0
        self.patience = random.randint(1, 15)
        
    def run(self):
        self.running = True
        global wait
        while(self.running): 
            if(coords[self.i][0] > coord_rect[2] ):
                if(coords[self.i][0] > coord_line[0]):
                    if(self.i==0):
                        self.bouger()
                    if(self.i>0 and coords[self.i][0] > coords[self.i-1][0]+25 ):
                        self.bouger()
                    else: 
                        self.attendre2()
                else:
                    i = 0  
                    if(self.comptoir == -1):
                        while(i != len(counter) and not counter[i]):
                            i = i + 1 
                        if(i != len(counter) and counter[i]): 
                            self.going = True 
                            self.comptoir = i
                            self.deplacement = (2*i - (nb_counter - 1))/nb_counter
                            counter[i] = False
                    if(self.going):
                        self.bouger()
                        fenetre.update()
                    else: 
                        self.attendre2()
                time.sleep(0.001)
                fenetre.after(1, self.run())
                
            if(coords[self.i][0] <= coord_rect[2]):
                self.traitement()
                
    def bouger(self):       
        coords[self.i] = (coords[self.i][0] - 1, coords[self.i][1] + self.deplacement)
        canvas.coords(c[self.i], coords[self.i][0], coords[self.i][1], coords[self.i][0]+20, coords[self.i][1]+20)   
        
    def arret(self): 
        self.running = False 
        
    def attendre(self): 
        wait[self.i] = wait[self.i] + 1 
        if(wait[self.i] > self.patience/2):
            if(wait[self.i] > (3*self.patience)/4):
                if(wait[self.i] > self.patience): 
                    canvas.itemconfigure(c[self.i], fill='black')
                else:
                    canvas.itemconfigure(c[self.i], fill='purple')
            else:
                canvas.itemconfigure(c[self.i], fill='violet')
    
    def attendre2(self): 
        wait[self.i] = wait[self.i] + self.patience
        if(wait[self.i] < len(color) - 1): 
            canvas.itemconfigure(c[self.i], fill=color[wait[self.i]%len(color)])
                
    def traitement(self): 
        canvas.itemconfigure(dispo[self.comptoir], fill='orange')
        counterTime = random.randint(1,timeInBank)
        time.sleep(counterTime)
        canvas.delete(fenetre, c[self.i])
        print("delete")
        coords[self.i] = (0,0)
        counter[self.comptoir] = True 
        canvas.itemconfigure(dispo[self.comptoir], fill='green')
        t[i].arret()

        
def apparition(i):
    if(i==0):
        c.append(canvas.create_rectangle(350,250,370,270,fill='#00FFFF'))
        coords.append((350, 250))
    elif(coords[len(coords) - 1][0] < 325):
        c.append(canvas.create_rectangle(350,250,370,270,fill='#00FFFF'))
        coords.append((350, 250))
    else: 
        c.append(canvas.create_rectangle(coords[len(coords) - 1][0] + 30,250,270,270,fill='#00FFFF'))
        coords.append((coords[len(coords) - 1][0] + 30, 250))
        
        
def source():
    time.sleep(2.0)
    for i in range(0,30): #fonctionne pour 3 pas pour 4 ou 5 
        apparition(i)
        fenetre.update()
            
        wait.append(0)
        t.append(myThread(i))
        t[i].start()
        TBA = random.randint(1, arrInt)
        time.sleep(TBA)
        
        
        
random.seed()  

canvas.pack()
draw = threading.Thread(target=source)
draw.start()


fenetre.mainloop()